In [1]:
from array import array
import time
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from numpy import random, arange
from scipy.signal import butter, sosfilt, sosfreqz, lfilter
import datetime
import random
from tkinter import *  
from tkinter.ttk import Progressbar  
from tkinter import ttk  
from tkinter import filedialog  
import sqlalchemy
from scipy import interpolate
import sys
import mysql
import mysql.connector
import pickle
import warnings
import xgboost as xgb
import lightgbm as lgbm
import catboost as catb
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
warnings.simplefilter('ignore')
import pathlib
import os
from forces import vagon_1
from forces import vagon_2
from forces import vagon_3
from forces import vagon_4


In [83]:
line = pd.read_excel('dannye.xlsx', sheet_name='line')
trains = pd.read_excel('dannye.xlsx', sheet_name='trains')
pod_sos = pd.read_excel('dannye.xlsx', sheet_name='pod_sos')

In [74]:
line

,vsp_type,vsp_cstr,vsp_cnd,rad_m,h_mm,i_prm,l_m,Shkol_mm,f_tr
0,1,1,1,10000,0,5,1000,1520,0.25
1,2,0,1,10000,0,0,2500,1525,0.25
2,1,1,1,5000,10,-5,1000,1515,0.35
3,1,0,1,10000,0,0,1000,1530,0.25
4,1,1,1,10000,0,0,1000,1520,0.25
5,3,1,1,2500,15,4,500,1520,0.20
6,1,1,1,10000,0,0,1000,1520,0.25
7,1,0,0,10000,0,1,1000,1520,0.25
8,5,1,0,350,100,0,300,1520,0.25
9,1,1,0,10000,0,0,1000,1520,0.40


In [13]:
trains

,mpoezda_min,mpoezda_max,poezd_gamma,meankver_force,meankgor_force,meankprd_force,meankram_force,rmskver_force,rmskgor_force,rmskprd_force,rmskram_force
0,0,5000,1,1,1,1,1,1,1,1,1


In [73]:
pod_sos

,vag_type,p_os,gamma,gamma_20,gamma_40,gamma_60,gamma_80,gamma_100,gamma_120,gamma_140,meankver_force,meankgor_force,meankprd_force,meankram_force,rmskver_force,rmskgor_force,rmskprd_force,rmskram_force
0,1,12,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,1,1,1,1,1,1,1,1
1,2,10,0.3,0.2,0.2,0.2,0.2,0.2,0.2,0.2,1,1,1,1,1,1,1,1
2,3,25,0.4,0.2,0.2,0.2,0.2,0.2,0.2,0.2,1,1,1,1,1,1,1,1
3,4,27,0.4,0.2,0.2,0.2,0.2,0.2,0.2,0.2,1,1,1,1,1,1,1,1


In [10]:
input_val = [2, 2, 250, 140, 80, 27.5, 1520, 0.25]
print ()
force = vagon_1 (input_val, 'return')
print (force)


[[61.8, 9.7, 13.350000000000001, 6.0], [79.1, 9.9, 19.3, 7.7], [44.5, 9.4, 7.4, 3.6]]


In [3]:
input_val = [2, 2, 250, 140, 80, 27.5, 1520, 0.25]
print ()
force = vagon_2 (input_val, 'return')
print (force)


[[137.4, 11.5, 38.2, 6.2], [185.4, 12.0, 53.7, 7.6], [89.5, 11.0, 22.6, 4.5]]


In [4]:
input_val = [2, 2, 250, 140, 80, 27.5, 1520, 0.25]
print ()
force = vagon_3 (input_val, 'return')
print (force)


[[109.35, 10.8, 22.25, 8.1], [135.6, 11.1, 32.7, 11.0], [83.1, 10.5, 11.8, 3.4]]


In [5]:
input_val = [2, 2, 250, 140, 80, 27.5, 1520, 0.25]
print ()
force = vagon_4 (input_val, 'return')
print (force)


[[120.5, 16.1, 25.65, 10.2], [155.3, 16.2, 38.5, 13.8], [85.7, 15.9, 12.8, 4.4]]


In [78]:
input_val = np.array([0, 0, 0,0,0,0,0,0.00])

In [79]:
print (input_val)

[0. 0. 0. 0. 0. 0. 0. 0.]


In [80]:
len(line.index)

13

In [ ]:
vsp_type	vsp_cstr	vsp_cnd	rad_m	h_mm	i_prm	l_m	Shkol_mm	f_tr
VSP_type = input_val[0] 
    Condition = input_val[1] 
    Radius = input_val[2]
    h = input_val[3]
    V = input_val[4]
    axle_load = input_val[5]
    Sh_Kol = input_val[6] 
    mu_fr = input_val[7] 
    vag_type	p_os	gamma	gamma_20	gamma_40	gamma_60	
    gamma_80	gamma_100	gamma_120	gamma_140	
    meankver_force	meankgor_force	meankprd_force	meankram_force	
    rmskver_force	rmskgor_force	rmskprd_force	rmskram_force


In [84]:
for index_line in range(len(line.index)):
    #print (index)
    input_val[0]= line['vsp_cstr'].loc[line['vsp_cstr'].index == index_line].values
    input_val[1]= line['vsp_cnd'].loc[line['vsp_cnd'].index == index_line].values
    input_val[2]= line['rad_m'].loc[line['rad_m'].index == index_line].values
    input_val[3]= line['h_mm'].loc[line['h_mm'].index == index_line].values
    input_val[6]= line['Shkol_mm'].loc[line['Shkol_mm'].index == index_line].values
    input_val[7]= (line['f_tr'].loc[line['f_tr'].index == index_line].values)
    for index_vagon in range(len(pod_sos.index)):
        gamma = pod_sos['gamma'].loc[pod_sos['gamma'].index == index_vagon].values
        p_os = pod_sos['p_os'].loc[pod_sos['p_os'].index == index_vagon].values
        vag_type = pod_sos['vag_type'].loc[pod_sos['vag_type'].index == index_vagon].values  
        meankver_force = pod_sos['meankver_force'].loc[pod_sos['meankver_force'].index == index_vagon].values  
        meankgor_force = pod_sos['meankgor_force'].loc[pod_sos['meankgor_force'].index == index_vagon].values  
        meankprd_force = pod_sos['meankprd_force'].loc[pod_sos['meankprd_force'].index == index_vagon].values  
        meankram_force = pod_sos['vag_type'].loc[pod_sos['vag_type'].index == index_vagon].values  
        rmskver_force = pod_sos['rmskver_force'].loc[pod_sos['rmskver_force'].index == index_vagon].values  
        rmskgor_force = pod_sos['rmskgor_force'].loc[pod_sos['rmskgor_force'].index == index_vagon].values  
        rmskprd_force = pod_sos['rmskprd_force'].loc[pod_sos['rmskprd_force'].index == index_vagon].values  
        rmskram_force = pod_sos['rmskram_force'].loc[pod_sos['rmskram_force'].index == index_vagon].values  
        for v in (10,30,50,70,90,110,130):
            input_val[4] = v
            input_val[5] = p_os
            print (input_val)
            print (vag_type)
            if vag_type == 1:
                
                force = vagon_1 (input_val, 'return')
                print (force)
            elif vag_type == 2:
                force = vagon_2 (input_val, 'return')
                print (force)     
            elif vag_type == 3:
                force = vagon_3 (input_val, 'return')
                print (force)
            elif vag_type == 4:
                force = vagon_4 (input_val, 'return')
                print (force)
            else:
                print ("Номер типа подвижного состава указан неверно или отстуствует в базе")

[1.00e+00 1.00e+00 1.00e+04 0.00e+00 1.00e+01 1.20e+01 1.52e+03 2.50e-01]
[1]
[[58.849999999999994, 8.0, 9.95, 6.5], [50.4, 8.3, 9.2, 7.8], [67.3, 7.6, 10.7, 4.9]]
[1.00e+00 1.00e+00 1.00e+04 0.00e+00 3.00e+01 1.20e+01 1.52e+03 2.50e-01]
[1]
[[58.900000000000006, 8.0, 9.8, 6.3], [50.4, 8.3, 9.0, 7.5], [67.4, 7.6, 10.6, 4.9]]
[1.00e+00 1.00e+00 1.00e+04 0.00e+00 5.00e+01 1.20e+01 1.52e+03 2.50e-01]
[1]
[[59.1, 7.9, 9.45, 6.0], [50.3, 8.2, 8.7, 7.0], [67.9, 7.6, 10.2, 4.7]]
[1.00e+00 1.00e+00 1.00e+04 0.00e+00 7.00e+01 1.20e+01 1.52e+03 2.50e-01]
[1]
[[59.400000000000006, 8.0, 9.0, 5.4], [50.1, 8.2, 8.3, 6.1], [68.7, 7.7, 9.7, 4.5]]
[1.00e+00 1.00e+00 1.00e+04 0.00e+00 9.00e+01 1.20e+01 1.52e+03 2.50e-01]
[1]
[[59.75, 8.0, 8.3, 4.5], [49.6, 8.1, 7.7, 4.9], [69.9, 7.9, 8.9, 4.1]]
[1.00e+00 1.00e+00 1.00e+04 0.00e+00 1.10e+02 1.20e+01 1.52e+03 2.50e-01]
[1]
[[60.25, 8.3, 7.45, 3.5], [49.2, 8.3, 6.9, 3.4], [71.3, 8.3, 8.0, 3.6]]
[1.00e+00 1.00e+00 1.00e+04 0.00e+00 1.30e+02 1.20e+01 1.52e+0

[0.00e+00 1.00e+00 1.00e+04 0.00e+00 1.00e+01 2.70e+01 1.53e+03 2.50e-01]
[5]
Номер типа подвижного состава указан неверно или отстуствует в базе
[0.00e+00 1.00e+00 1.00e+04 0.00e+00 3.00e+01 2.70e+01 1.53e+03 2.50e-01]
[5]
Номер типа подвижного состава указан неверно или отстуствует в базе
[0.00e+00 1.00e+00 1.00e+04 0.00e+00 5.00e+01 2.70e+01 1.53e+03 2.50e-01]
[5]
Номер типа подвижного состава указан неверно или отстуствует в базе
[0.00e+00 1.00e+00 1.00e+04 0.00e+00 7.00e+01 2.70e+01 1.53e+03 2.50e-01]
[5]
Номер типа подвижного состава указан неверно или отстуствует в базе
[0.00e+00 1.00e+00 1.00e+04 0.00e+00 9.00e+01 2.70e+01 1.53e+03 2.50e-01]
[5]
Номер типа подвижного состава указан неверно или отстуствует в базе
[0.00e+00 1.00e+00 1.00e+04 0.00e+00 1.10e+02 2.70e+01 1.53e+03 2.50e-01]
[5]
Номер типа подвижного состава указан неверно или отстуствует в базе
[0.00e+00 1.00e+00 1.00e+04 0.00e+00 1.30e+02 2.70e+01 1.53e+03 2.50e-01]
[5]
Номер типа подвижного состава указан неверно и

[[61.800000000000004, 7.5, 11.350000000000001, 5.1], [69.9, 7.5, 13.8, 6.3], [53.7, 7.4, 8.9, 3.6]]
[1.00e+00 0.00e+00 3.50e+02 1.00e+02 9.00e+01 1.20e+01 1.52e+03 2.50e-01]
[1]
[[61.95, 9.1, 12.55, 6.5], [79.7, 9.2, 18.8, 8.5], [44.2, 9.0, 6.3, 3.4]]
[1.00e+00 0.00e+00 3.50e+02 1.00e+02 1.10e+02 1.20e+01 1.52e+03 2.50e-01]
[1]
[[62.0, 12.2, 15.15, 9.1], [90.7, 12.2, 25.8, 12.4], [33.3, 12.1, 4.5, 3.4]]
[1.00e+00 0.00e+00 3.50e+02 1.00e+02 1.30e+02 1.20e+01 1.52e+03 2.50e-01]
[1]
[[62.05, 15.7, 19.1, 11.7], [103.7, 16.0, 35.2, 16.3], [20.4, 15.4, 3.0, 3.0]]
[1.00e+00 0.00e+00 3.50e+02 1.00e+02 1.00e+01 1.00e+01 1.52e+03 2.50e-01]
[2]
[[51.8, 1.8, 9.3, 1.5], [35.6, 1.7, 4.3, 1.4], [67.9, 1.8, 14.3, 1.5]]
[1.00e+00 0.00e+00 3.50e+02 1.00e+02 3.00e+01 1.00e+01 1.52e+03 2.50e-01]
[2]
[[51.2, 3.4, 8.8, 1.9], [37.0, 3.3, 4.4, 1.9], [65.4, 3.4, 13.2, 1.9]]
[1.00e+00 0.00e+00 3.50e+02 1.00e+02 5.00e+01 1.00e+01 1.52e+03 2.50e-01]
[2]
[[50.7, 3.8, 8.9, 2.0], [40.3, 3.7, 6.0, 2.2], [61.1, 3.9, 1

[[48.2, 7.2, 2.2, 3.8], [46.6, 7.2, 1.9, 3.8], [49.9, 7.1, 2.5, 3.8]]
[1.00e+00 1.00e+00 1.00e+04 0.00e+00 1.10e+02 1.00e+01 1.52e+03 2.50e-01]
[2]
[[47.7, 9.1, 2.3, 4.6], [47.7, 9.1, 2.3, 4.6], [47.6, 9.1, 2.3, 4.5]]
[1.00e+00 1.00e+00 1.00e+04 0.00e+00 1.30e+02 1.00e+01 1.52e+03 2.50e-01]
[2]
[[47.7, 9.1, 2.3, 4.6], [47.7, 9.1, 2.3, 4.6], [47.6, 9.1, 2.3, 4.5]]
[1.00e+00 1.00e+00 1.00e+04 0.00e+00 1.00e+01 2.50e+01 1.52e+03 2.50e-01]
[3]
[[88.95, 21.7, 56.650000000000006, 16.5], [95.7, 23.0, 74.9, 17.7], [82.2, 20.3, 38.4, 15.2]]
[1.00e+00 1.00e+00 1.00e+04 0.00e+00 3.00e+01 2.50e+01 1.52e+03 2.50e-01]
[3]
[[91.3, 18.3, 54.85, 15.9], [101.5, 19.6, 72.9, 17.0], [81.1, 17.0, 36.8, 14.7]]
[1.00e+00 1.00e+00 1.00e+04 0.00e+00 5.00e+01 2.50e+01 1.52e+03 2.50e-01]
[3]
[[94.95, 13.4, 51.9, 14.7], [110.0, 14.5, 69.6, 15.8], [79.9, 12.1, 34.2, 13.5]]
[1.00e+00 1.00e+00 1.00e+04 0.00e+00 7.00e+01 2.50e+01 1.52e+03 2.50e-01]
[3]
[[99.75, 8.1, 48.05, 13.5], [120.5, 9.2, 65.3, 14.7], [79.0, 6.9, 

In [ ]:
df2 = df1.copy()
with pd.ExcelWriter('output.xlsx') as writer:  
    df1.to_excel(writer, sheet_name='Sheet_name_1')
    df2.to_excel(writer, sheet_name='Sheet_name_2')
    
#ExcelWriter также можно использовать для добавления в существующий файл Excel:

with pd.ExcelWriter('output.xlsx',
                    mode='a') as writer:  
    df.to_excel(writer, sheet_name='Sheet_name_3')